In [1]:
# create all models trained until a given date
import make_model as tm
import pandas as pd
import datetime
import os

# 

last_trains = [#'2021-03-15', '2021-03-31', '2021-04-15', '2021-04-30', 
               #'2021-05-15', '2021-05-31', '2021-06-15', '2021-06-30',
               '2021-07-15']#, '2021-07-31', '2021-08-15', '2021-08-31',
               #'2021-09-15', 
               #'2021-09-30', '2021-10-15', '2021-10-31',
               #'2021-11-15', '2021-11-30']
#   
start_times = [#'2021/03/16/09:00', '2021/04/01/09:00',
               #'2021/04/16/09:00', '2021/05/01/09:00', 
               #'2021/05/16/09:00', '2021/06/01/09:00',
               #'2021/06/16/09:00', '2021/07/01/09:00',
               '2021/07/16/09:00']#, '2021/08/01/09:00',
               #'2021/08/16/09:00', '2021/09/01/09:00',
               #'2021/09/16/09:00', 
               #'2021/10/01/09:00', '2021/10/16/09:00', 
               #'2021/11/01/09:00',
               #'2021/11/16/09:00', '2021/12/01/09:00']
# 
end_times = [  #'2021/03/31/15:00', '2021/04/15/15:00',
               #'2021/04/30/15:00', '2021/05/15/15:00',
               #'2021/05/31/15:00', '2021/06/15/15:00',
               #'2021/06/30/15:00', '2021/07/15/15:00',
               '2021/07/31/15:00']#, 
               #'2021/08/15/15:00', '2021/08/31/15:00', 
               #'2021/09/15/15:00', '2021/09/30/15:00', 
               #'2021/10/15/15:00', '2021/10/31/15:00', 
               #'2021/11/15/15:00', '2021/11/30/15:00',
               #'2021/12/09/15:00']

def para_setup(model_name, tm):
    c = model_name.find('C')
    h = model_name.find('HL')
    p = model_name.find('P')
    if c != -1:
        tm.pred_term = int(model_name[:c])
        tm.target_type = 'C'
        tm.base1 = '종가'
        tm.base2 = '종가'
    elif h != -1:
        tm.pred_term = int(model_name[:h])
        tm.target_type = 'HL'
        tm.base1 = '고가'
        tm.base2 = '저가'
    elif p != -1:
        tm.pred_term = int(model_name[:p])
        tm.target_type = 'P'
        tm.base1 = '종가'
        tm.base2 = '종가'
    else:
        print("argument error " + model_name)
        exit(0)

    if h != -1:
        tm.checkpoint_path_best = tm.last_train+"/" + "60M_" + str(tm.pred_term) + "HL_best"
    elif c != -1:
        tm.checkpoint_path_best = tm.last_train+"/" + "60M_" + str(tm.pred_term) + "C_best"
    elif p != -1:
        tm.checkpoint_path_best = tm.last_train+"/" + "60M_" + str(tm.pred_term) + "P_best"
    else:
        print("argument error")
        exit(1)

    model = tm.create_model(3)
    
    return model, tm

for i in range(len(last_trains)):
    tm.last_train = last_trains[i]
    tm.start_time = start_times[i]
    tm.end_time = end_times[i]
    tm.max_repeat_cnt = 100

    tm.checkpoint_path = last_trains[i]+"/60M_input83_test"
    
    tm.df_raw_path = tm.last_train+'/kospi200f_60M_raw.csv'
    tm.norm_df_path = tm.last_train+'/kospi200f_60M_norm.csv'
    tm.df_pred_path = tm.last_train+'/kospi200f_60M_pred.csv'
    tm.result_path = tm.last_train+'/pred_83_results.csv'
    
    models = ['5C', '5HL', '5P', '10C', '10HL', '10P', '15C', '15HL', '15P', '20C', '20HL', '20P',  
              '25C', '25HL', '25P', '30C', '30HL', '30P', '40C', '40HL', '40P']
    profits = []
    for model in models:

        created_model, tm = para_setup(model, tm)    

        print('data processing start...')
        now = datetime.datetime.now()
        print(now)
        tm.preprocessing()
        print('data processing end...')

        tm.make_train_data()

        now = datetime.datetime.now()
        print(now)
        tm.train(created_model, 3)
        print('training end...')

        r = tm.predict(created_model)
        #print(r)

        import profit
        profit.pred_term = tm.pred_term
        profit.result_path = tm.result_path
        p = profit.calc_profit()
        print(model + " 수익률: " + str(p))
        
        profits.append(p)

    print("=========================================================================")
    print(last_trains[i] + " training end")
    print("=========================================================================")
    dic = {'model': models, tm.start_time[5:10] + '~' + tm.end_time[5:10]: profits}
    file_dir = tm.last_train
    try:
        if not os.path.isdir(file_dir):
            os.makedirs(file_dir)
    except:
        print('the directory is not made ..' + dir)
        exit(1)
    pd.DataFrame(dic).to_csv(file_dir + "/models.csv", index=False, encoding='euc-kr')

data processing start...
2021-12-12 12:51:17.161708
data processing end...
train을 위한 _norm.csv 파일 생성 완료
2021-12-12 12:51:23.117300
best accuracy 0.2962962962962963
best accuracy 0.5079365079365079
best accuracy 0.5932203389830508
반복 횟수 : 100 accuracy = 0.375
best accuracy 0.5932203389830508
training end...
5C 수익률: 0.9203502575766433
data processing start...
2021-12-12 12:51:54.669300
data processing end...
train을 위한 _norm.csv 파일 생성 완료
2021-12-12 12:52:00.519300
best accuracy 0.6290322580645161
best accuracy 0.6666666666666666
best accuracy 0.7037037037037037
best accuracy 0.7407407407407407
반복 횟수 : 100 accuracy = 0.3382352941176471
best accuracy 0.7407407407407407
training end...
5HL 수익률: 0.5798216364378629
data processing start...
2021-12-12 12:52:42.884733
data processing end...
train을 위한 _norm.csv 파일 생성 완료
2021-12-12 12:52:47.512732
best accuracy 0.6333333333333333
best accuracy 0.7272727272727273
반복 횟수 : 100 accuracy = 0.5
best accuracy 0.7272727272727273
training end...
5P 수익률: 1.

In [ ]:
# create all ensembles
import create_all_ensembles as ce
import os

last_trains = ['2021-03-15', '2021-03-31', '2021-04-15', '2021-04-30', 
               '2021-05-15', '2021-05-31', '2021-06-15', '2021-06-30',
               '2021-07-15', '2021-07-31', '2021-08-15', '2021-08-31',
               '2021-09-15', '2021-09-30', '2021-10-15', '2021-10-31',
               '2021-11-15', '2021-11-30']
#   
start_times = ['2021/03/16/09:00', '2021/04/01/09:00',
               '2021/04/16/09:00', '2021/05/01/09:00', 
               '2021/05/16/09:00', '2021/06/01/09:00',
               '2021/06/16/09:00', '2021/07/01/09:00',
               '2021/07/16/09:00', '2021/08/01/09:00',
               '2021/08/16/09:00', '2021/09/01/09:00',
               '2021/09/16/09:00', '2021/10/01/09:00',
               '2021/10/16/09:00', '2021/11/01/09:00',
               '2021/11/16/09:00', '2021/12/01/09:00']
# 
end_times = [  '2021/03/31/15:00', '2021/04/15/15:00',
               '2021/04/30/15:00', '2021/05/15/15:00',
               '2021/05/31/15:00', '2021/06/15/15:00',
               '2021/06/30/15:00', '2021/07/15/15:00',
               '2021/07/31/15:00'  '2021/08/15/15:00',
               '2021/08/31/15:00', '2021/09/15/15:00',
               '2021/09/30/15:00', '2021/10/15/15:00',
               '2021/10/31/15:00', '2021/11/15/15:00',
               '2021/11/30/15:00', '2021/12/09/15:00']


for i in range(len(last_trains)):
    ce.ep.last_train = last_trains[i]

    ce.ep.start_time = start_times[i]
    ce.ep.end_time = end_times[i]

    ce.ep.df_pred_path = ce.ep.last_train+'/kospi200f_60M_pred.csv'
    ce.ep.result_path = ce.ep.last_train+'/pred_83_results.csv'

    ce.create_ensembles()

2021-12-12 14:35:16.617123
nothing done! in this preprocessing
[1, 3, 5, 1.7299005666405414]
 
0
 
[1, 4, 18, 1.8869825034661574]
[13, 8, 9, 1.722672584242566]
[20, 14, 5, 1.2306743926698342]
[14, 3, 1, 1.3821555247453081]
[1, 6, 14, 1.8018054282958593]
[5, 1, 19, 1.6797492012779522]
[13, 9, 10, 1.8014255982880225]
[4, 1, 6, 1.6929392368436917]
[9, 12, 20, 1.5301725842425724]
[9, 10, 16, 1.339125730906023]
[19, 18, 6, 1.4898463288926393]
[14, 12, 2, 1.5505476068479034]
[9, 21, 2, 1.4201759901139321]
[7, 5, 1, 1.5904597926336734]
[11, 20, 6, 1.6985632497438083]
[5, 16, 17, 1.5147242600518367]
[9, 7, 5, 1.8029315811682447]
[3, 20, 10, 1.8308166706853921]
[3, 8, 14, 1.7211832991741458]
[8, 9, 20, 1.4717391192959193]
[3, 6, 7, 1.7584850201941056]
[1, 12, 20, 1.7434147025137166]
[4, 7, 17, 1.5505080173609]
[2, 13, 5, 1.681277382602922]
[1, 15, 18, 1.8112383356441029]
[16, 21, 15, 1.2352519289890902]
[14, 11, 12, 1.6334276930496046]
[14, 9, 11, 1.52453042678883]
[16, 14, 15, 1.38548737114955

[9, 2, 5, 1.5139903550545477]
[16, 8, 17, 1.4197755440351982]
[6, 13, 14, 2.068931701730057]
[13, 20, 5, 1.633035927421782]
[3, 5, 2, 1.403615015974435]
[13, 3, 10, 1.6854048315148589]
[8, 12, 5, 1.5900798872746957]
[1, 10, 9, 1.6978055639279046]
[7, 13, 9, 1.9144551962143592]
[5, 9, 17, 1.379846524805588]
[8, 6, 18, 2.0794684128036685]
[3, 1, 15, 1.8368759870998894]
[8, 18, 10, 1.5915652239435791]
[7, 11, 1, 1.8345967508590042]
[10, 21, 19, 1.1430118451986226]
[20, 21, 11, 1.4333360811381042]
[6, 20, 4, 1.672564364940625]
[3, 2, 17, 1.1783165501235737]
[12, 17, 8, 1.701219467719568]
[1, 12, 6, 1.8289335101573365]
[19, 3, 6, 1.7147730273072528]
[21, 13, 20, 1.6352118873952617]
[20, 6, 5, 1.8458767707517048]
[16, 20, 14, 1.642430104285971]
[18, 8, 15, 1.8462615287238502]
[7, 17, 11, 1.5350752908553826]
[12, 16, 7, 1.4054471788534582]
[4, 19, 21, 1.146343164144912]
[6, 3, 4, 1.9073139731147188]
[9, 14, 4, 1.759593134004462]
[11, 12, 18, 2.0998156459099384]
[10, 6, 20, 1.486074763397434]


[11, 4, 18, 2.0805991470251373]
[19, 18, 8, 1.5075558502622224]
[3, 18, 12, 1.8089763698836585]
[16, 8, 4, 1.750944797757546]
[12, 14, 4, 1.615686057025741]
 
500
 
[17, 20, 11, 1.3960564380010845]
[2, 7, 18, 1.6838832057387383]
[3, 18, 14, 1.750959762493216]
[14, 8, 6, 1.7754362981493723]
[20, 3, 18, 1.8364713213575274]
[17, 19, 13, 1.376074552414245]
[16, 12, 17, 1.4498947344625948]
[2, 16, 11, 1.7683015401772209]
[14, 19, 16, 1.4928684369160226]
[14, 4, 17, 1.6714610585327578]
[14, 8, 1, 1.5667143589125259]
[15, 6, 1, 1.9099395683886926]
[5, 19, 10, 1.3350117246368065]
[16, 15, 2, 1.6435632798842592]
[17, 8, 9, 1.683130131412374]
[8, 16, 6, 1.4532945626620064]
[17, 19, 4, 1.267925764060521]
[11, 5, 4, 1.6955883567424137]
[10, 4, 18, 2.013915305322807]
[16, 6, 1, 1.5075593766954025]
[9, 17, 18, 1.7837115558502603]
[2, 1, 17, 1.4254637108927544]
[19, 7, 14, 1.6394355295677807]
[12, 17, 5, 1.503040810175412]
[7, 11, 16, 1.340650852974861]
[4, 18, 8, 2.1363537434444506]
[12, 21, 3, 1.61

[2, 1, 21, 1.574647853999636]
[7, 21, 20, 1.29807092048948]
[8, 1, 20, 1.6714353939357403]
[16, 17, 2, 1.4875972632467274]
[3, 18, 4, 1.8877243052625263]
[1, 4, 19, 1.8594755410211572]
[6, 7, 11, 1.703844595816507]
[19, 5, 20, 1.400944255229371]
[19, 8, 7, 1.018531165229969]
[16, 21, 14, 1.4122367382000083]
[19, 1, 6, 1.4996425944903253]
[18, 13, 14, 1.8918486798480871]
[19, 14, 11, 1.6081741967568814]
[10, 4, 6, 1.8033110796310816]
[1, 12, 14, 1.6582612574597633]
[6, 8, 21, 1.6130453161733715]
[5, 18, 17, 1.4857138163843449]
[4, 17, 15, 1.7354985532581841]
[5, 9, 8, 1.3858777352462444]
[2, 16, 20, 1.7859772138163843]
[7, 15, 20, 1.6100342998372428]
[14, 16, 9, 1.495109711254443]
[14, 5, 10, 1.2449819006570544]
[12, 1, 17, 1.9381865694134692]
[12, 21, 16, 1.127747453131593]
[12, 18, 1, 1.773564349870401]
[19, 5, 6, 1.5550137440472591]
[14, 10, 1, 1.576879362830789]
[5, 3, 4, 2.096823407076978]
[12, 2, 19, 1.2717250437036607]
[6, 13, 8, 2.260320875278796]
[4, 6, 17, 1.7132747905238421]


[19, 12, 17, 1.4661034269696798]
[10, 13, 12, 1.9050285430104286]
[10, 8, 3, 1.792376529628063]
[13, 17, 8, 1.602510051841575]
[19, 21, 13, 1.4650937217433173]
[8, 12, 13, 1.7822133492072987]
[16, 5, 3, 1.4518026553740375]
[21, 7, 2, 1.7437919103020088]
 
1000
 
[20, 12, 18, 1.9494911537766004]
[1, 2, 19, 1.6168615347519393]
[1, 7, 17, 1.8398601784314899]
[21, 3, 7, 1.818015974440894]
[14, 12, 15, 1.6296241485321623]
[3, 11, 7, 2.1145278497799693]
[4, 7, 8, 1.4438831153173803]
[10, 5, 6, 1.826287961902468]
[3, 20, 14, 1.7535907679787757]
[7, 16, 4, 0.9284313551148355]
[2, 20, 14, 1.1493003646994928]
[3, 2, 4, 1.7528699439387512]
[18, 13, 21, 1.7049802278618338]
[6, 15, 10, 1.7697778497799763]
[19, 13, 15, 1.7785997649044527]
[12, 10, 8, 1.5520031798179534]
[20, 11, 16, 1.7008347097474243]
[4, 19, 5, 1.442386144433056]
[6, 5, 7, 1.7437821297245173]
[10, 15, 20, 1.6684228404364343]
[10, 8, 1, 1.830420308638256]
[9, 6, 7, 1.8854404123214183]
[18, 2, 10, 1.6676761408162042]
[13, 9, 18, 1.7

...

[3, 10, 9, 1.3912320595357603]
[20, 16, 12, 1.2339668212155241]
[1, 17, 13, 1.2354407890851111]
[7, 11, 18, 1.1892909013053004]
[2, 13, 6, 1.0497484495895097]
[1, 8, 16, 1.4004437717795741]
[14, 11, 12, 1.3779179463705646]
[7, 18, 3, 1.1904183598133575]
[2, 10, 15, 1.351737700076781]
[14, 18, 20, 1.2823068926820624]
[3, 5, 10, 1.1686391677987114]
[2, 18, 3, 1.275658451952041]
[2, 20, 4, 1.1542083161065515]
[7, 13, 9, 1.1649102829130018]
[4, 10, 3, 1.4178093290414004]
[11, 15, 9, 1.3842035024511248]
[13, 5, 6, 1.095600732384386]
[14, 4, 7, 1.1789453074242546]
[8, 1, 2, 1.2483456677101186]
[1, 21, 4, 1.2919006260705137]
[17, 5, 1, 1.265344781761264]
[17, 11, 3, 1.3402726655247734]
[10, 21, 2, 1.2852937068099888]
[13, 1, 3, 1.292274895162715]
[20, 14, 8, 1.1789477437835922]
[18, 13, 21, 1.0929227304943554]
[1, 14, 13, 1.35944405233005]
[2, 18, 7, 1.3535658407654625]
[21, 18, 7, 1.2472372423365363]
[18, 17, 8, 1.0103433199456628]
[3, 11, 16, 1.2561137263008688]
[21, 9, 8, 1.32404409072116]

[11, 19, 8, 1.3731117624475822]
[3, 8, 9, 1.1682717205126698]
[20, 8, 5, 1.3587230966865538]
[3, 2, 7, 1.3376917783946563]
[21, 3, 2, 1.2826874520111005]
[4, 18, 11, 1.432927854231883]
[1, 15, 10, 1.131356210501446]
[11, 20, 18, 1.2893207577815842]
[13, 19, 15, 1.2346883970232163]
[11, 13, 18, 1.1405383025220044]
[8, 6, 21, 1.0590150168330292]
[6, 16, 1, 1.2510064821924272]
[16, 2, 19, 1.4362385564940034]
[7, 5, 12, 1.0220832496603873]
[6, 9, 11, 1.4100352902959044]
[20, 10, 7, 1.26977634221251]
[21, 3, 10, 1.2483794371271602]
[16, 1, 13, 1.248051591754771]
[15, 9, 19, 1.3122248390526288]
[17, 1, 3, 1.2225256482192375]
[1, 7, 9, 1.4580306242986252]
[3, 7, 12, 1.155353611718151]
[10, 1, 17, 1.1475958596657012]
[12, 11, 10, 1.2694734658319073]
[19, 2, 10, 1.3974885269623742]
[19, 14, 5, 1.3243841769535203]
[18, 11, 3, 1.3727628167267145]
[19, 2, 3, 1.2649423099639678]
 
400
 
[12, 15, 7, 1.1912153151024767]
[16, 1, 11, 1.3642548874844946]
[19, 2, 21, 1.298911611836274]
[1, 9, 12, 1.42184

[21, 10, 20, 1.1805007235248948]
[16, 6, 17, 1.1775505729135893]
[16, 21, 8, 1.2040850510897192]
[8, 21, 4, 1.201469759612545]
[18, 2, 8, 1.1313190301813256]
[1, 11, 14, 1.338047856003781]
[17, 21, 6, 1.1915164638828162]
[7, 12, 1, 1.1811840706396555]
[14, 16, 13, 1.1325008268855985]
[13, 6, 12, 1.0892668920914312]
[15, 2, 21, 1.2993116915716718]
[20, 10, 14, 1.338047856003781]
[13, 1, 5, 1.3937825290886576]
[16, 17, 10, 1.1915230642017611]
[15, 14, 1, 1.2546333353021055]
[3, 9, 2, 1.2487180763097303]
[5, 11, 9, 1.1183930807394704]
[17, 12, 14, 1.1819804057645773]
[5, 4, 3, 0.8511489280018945]
[2, 16, 18, 1.373870533341211]
[5, 19, 6, 1.012156680054337]
[17, 6, 20, 1.090428503927706]
[8, 1, 11, 1.3302953753469975]
[11, 8, 4, 1.4498969641486061]
[11, 8, 6, 1.264980258106433]
[6, 7, 9, 1.2572239235721434]
[20, 3, 6, 1.1948437629200857]
[8, 17, 21, 1.0103433199456628]
[17, 18, 21, 1.2095927145472738]
[12, 11, 5, 1.1996248006615082]
[16, 21, 11, 1.0959638385210566]
[11, 17, 12, 1.174596302

[11, 17, 7, 1.1897352341858147]
[6, 7, 12, 0.8634159677514615]
[5, 15, 10, 1.3657599816903907]
[14, 12, 13, 1.1682409042584652]
[16, 15, 11, 1.3060159027818823]
[11, 15, 19, 1.2092133955466324]
[18, 16, 14, 1.3044632774201204]
[19, 11, 21, 1.1811603862737]
[16, 9, 6, 1.1461137558324976]
[6, 16, 11, 1.07124388695293]
[19, 17, 11, 1.2033758933317618]
[2, 18, 6, 1.001027287224616]
[19, 11, 13, 1.2136453989723006]
[17, 20, 1, 1.3063113371921315]
[14, 19, 2, 1.3103522089658006]
[7, 4, 10, 1.3513655572618277]
[19, 13, 8, 1.2660742425137335]
[5, 4, 6, 0.9076367905026252]
[16, 17, 3, 1.001108233417993]
[15, 9, 14, 1.1601338521056075]
[20, 7, 9, 1.3417513289232768]
[4, 13, 6, 1.1590204801842832]
[10, 7, 16, 1.1472903697359857]
[8, 11, 14, 1.2749140777272463]
[8, 10, 21, 1.1686477615025683]
[3, 18, 16, 1.188946695410782]
[21, 15, 7, 1.1254491317701267]
[14, 21, 15, 1.0342747770362062]
[14, 15, 10, 1.376093570964504]
[15, 6, 11, 1.2668320388636276]
[15, 5, 6, 0.893986002008153]
 
900
 
[14, 17, 9

[3, 9, 21, 1.2210464089539856]
[16, 8, 3, 1.423346125450359]
[5, 16, 11, 1.2941334091311805]
[16, 6, 15, 1.2310152087886126]
[12, 15, 3, 1.128401364361231]
[20, 7, 12, 1.2217746736755095]
[7, 16, 18, 1.1978479564113151]
[16, 1, 18, 1.2228779162482986]
[15, 11, 3, 1.3347491140511505]
[6, 10, 1, 1.2454279132951387]
[1, 12, 15, 1.2033314186994257]
[21, 19, 16, 0.9984335981336004]
[20, 15, 6, 1.2901014411434644]
[21, 8, 1, 1.3059422066032658]
[8, 19, 12, 1.1575401777804049]
[20, 9, 1, 1.215863608174353]
[19, 21, 9, 1.0749149932077264]
[3, 13, 16, 1.1528242868111782]
[13, 5, 8, 1.4266428740180777]
[15, 11, 7, 1.1745968785068843]
[14, 8, 5, 1.1726704713247909]
[8, 3, 15, 1.2694259789734805]
[21, 19, 3, 1.2841673114405516]
[9, 11, 18, 1.206641057822928]
[20, 11, 7, 1.338047856003781]
[20, 16, 8, 1.393807985352314]
[6, 3, 10, 1.2960052418640358]
[12, 4, 8, 1.3956405557852518]
[5, 16, 21, 1.0077426023270946]
[12, 21, 7, 1.1191662335361185]
[19, 4, 9, 1.233192560982817]
[4, 2, 9, 1.0312743783592

In [ ]:
et.selected_num = 3
et.weights = [1 for i in range(et.selected_num)]
#et.weights = [0.34666667, 0.32833333, 0.325]

et.selected_checkpoint_path = ["", et.checkpoint_path[3], et.checkpoint_path[11], et.checkpoint_path[14]]

In [ ]:
print("preprocessing start......")
et.preprocessing()
print("preprocessing end........")

# type=0 --> 0: 중립 1:고점 2:저점,  type=1 --> n일 후 0:하락  1: 상승
r = et.predict()
print("수익률: " + str(r))